## 주제 
- 위성 소형화, 저궤도 위성, 광학위성

In [27]:
"""
담당자별 에이전트:
- 현준: startup_search_agent (스타트업 탐색)
- 승연: tech_summary_agent (기술 요약)
- 민주: market_eval_agent (시장성 평가)
- 경남: investor_insight_agent (투자자 인사이트)

+ investment_decision_agent (투자 판단)
"""

'\n담당자별 에이전트:\n- 현준: startup_search_agent (스타트업 탐색)\n- 승연: tech_summary_agent (기술 요약)\n- 민주: market_eval_agent (시장성 평가)\n- 경남: investor_insight_agent (투자자 인사이트)\n\n+ investment_decision_agent (투자 판단)\n'

In [28]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
from typing import Annotated, TypedDict, Sequence, Literal
from pydantic import BaseModel

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_react_agent

from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [30]:
class AgentState(TypedDict):
    startup_name: str
    startup_info: dict
    tech_summary: str
    market_analysis: str
    investment_decision: str
    report: str
    iteration_count: int


## 스타트업 탐색 에이전트 (현준)
1. 웹서치로 스타트업 기본 정보 수집


In [ ]:
def startup_search_agent(state: AgentState) -> AgentState:
    # TODO: web_search 또는 web_fetch로 스타트업 정보 수집

    return state

## 기술 요약 에이전트 (승연)

In [ ]:
def tech_summary_agent(state: AgentState) -> AgentState:
    """
    스타트업의 기술력 핵심 요약
    - RAG: 홈페이지, 논문에서 핵심 기술 추출
    """
    return state

## 시장성 평가 에이전트 (민주)

In [33]:
def market_eval_agent(state: AgentState) -> AgentState:
    """
    시장 성장성, 수요 분석
    - RAG: 시장 리포트, 산업뉴스 검색
    """
    return state

## 유명 투자자 지식 기반 가치 요약 에이전트

In [ ]:
def investment_decision_agent(state: AgentState) -> AgentState:
    """
    유명 투자자 지식 기반 가치 요약 에이전트
    - Scorecard Method 적용
    """

    return state

## 투자 판단 에이전트

In [ ]:
def investment_decision_agent(state: AgentState) -> AgentState:
    """
    종합 판단 (리스크, ROI 등)
    - Scorecard Method 적용
    """
    print("=== [경남] 투자 판단 에이전트 실행 ===")

    company_name = state["startup_name"]
    tech_summary = state["tech_summary"]
    market_analysis = state["market_analysis"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    evaluation_prompt = f"""
    다음 정보를 바탕으로 Scorecard Method로 투자를 평가하세요:
    
    회사: {company_name}
    
    기술 분석:
    {tech_summary}
    
    시장 분석:
    {market_analysis}
    
    평가 기준:
    - 제품/기술: 15-20%
    - 시장 기회: 20-25%
    
    각 항목을 50-150% 척도로 평가하고, 
    "invest" 또는 "hold" 중 하나를 결정하세요.
    
    형식:
    기술 점수: X%
    시장 점수: Y%
    결론: invest/hold
    """

    result = llm.invoke(evaluation_prompt)
    evaluation_text = result.content

    print(f"평가 결과:\n{evaluation_text}")

    if "invest" in evaluation_text.lower() and "hold" not in evaluation_text.lower():
        state["investment_decision"] = "invest"
    else:
        state["investment_decision"] = "hold"

    state["iteration_count"] = state.get("iteration_count", 0) + 1

    print(f"투자 판단: {state['investment_decision']}")
    return state


def should_continue(state: AgentState) -> str:
    """조건 분기: 투자 추천이면 report_writer, 보류면 startup_search"""
    if state["investment_decision"] == "invest":
        return "report_writer"
    elif state["iteration_count"] >= 3:
        return "report_writer"
    return "startup_search"

## 보고서 생성 에이전트

In [37]:
def report_writer_agent(state: AgentState) -> AgentState:
    """
    결과 요약 보고서 생성
    """
    print("=== 보고서 생성 에이전트 실행 ===")

    company_name = state["startup_name"]
    company_info = state["startup_info"]
    tech_summary = state["tech_summary"]
    market_analysis = state["market_analysis"]
    investment_decision = state["investment_decision"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    report_prompt = f"""
    당신은 전문 벤처 투자 분석가입니다.
    아래 분석 결과를 바탕으로 다음 형식에 맞춰 투자 평가 보고서를 작성하세요.
    
    === 분석 정보 ===
    
    회사명: {company_name}
    회사 정보: {company_info}
    
    기술력 분석:
    {tech_summary}
    
    시장성 분석:
    {market_analysis}
    
    
    투자 판단: {investment_decision}
    
    === 보고서 형식 ===
    
    # 위성 스타트업 투자 평가 보고서
    
    ## Executive Summary
    
    ## 1. 회사 개요
    
    ## 2. 기술력 평가
    ### 2.1 핵심 기술
    ### 2.2 기술적 강점
    ### 2.3 기술 리스크
    ### 2.4 Scorecard 평가
    
    ## 3. 시장성 분석
    ### 3.1 시장 규모 및 성장률
    ### 3.2 시장 기회
    ### 3.3 Scorecard 평가
    
    ## 4. 경쟁 환경
    
    ## 5. 투자 의견
    ### 5.1 최종 판단
    ### 5.2 투자 근거
    ### 5.3 주요 리스크
    
    ## 6. 결론
    """

    result = llm.invoke(report_prompt)
    state["report"] = result.content

    print("보고서 생성 완료")
    return state

## LangGraph 구성

In [ ]:
def create_workflow():
    graph = StateGraph(AgentState)

    # 노드 추가
    graph.add_node("startup_search", startup_search_agent)  # 현준
    graph.add_node("tech_summary", tech_summary_agent)  # 승연
    graph.add_node("market_eval", market_eval_agent)  # 민주
    graph.add_node("investor_insight", investor_insight_agent)  # 경남
    graph.add_node("investment_decision", investment_decision_agent)  # 경남
    graph.add_node("report_writer", report_writer_agent)

    # 시작점
    graph.set_entry_point("startup_search")

    # Edge 연결
    graph.add_edge("startup_search", "tech_summary")
    graph.add_edge("tech_summary", "market_eval")
    graph.add_edge("market_eval", "investor_insight")
    graph.add_edge("investor_insight", "investment_decision")

    # 조건부 Edge (Reflection)
    graph.add_conditional_edges(
        "investment_decision",
        should_continue,
        {"report_writer": "report_writer", "startup_search": "startup_search"},
    )

    # 종료점
    graph.set_finish_point("report_writer")

    return graph.compile()